In [1]:
# Step 1: Download the file
!wget https://indic-asr-public.objectstore.e2enetworks.net/lahaja_artifacts/lahaja.tgz

# Step 2: Extract the .tgz file
!tar -xvzf lahaja.tgz

Streaming output truncated to the last 5000 lines.
lahaja/audios/5348024557504419_f738_chunk_1_subchunk_1.wav
lahaja/audios/5348024557508431_f1391_chunk_0_subchunk_1.wav
lahaja/audios/5348024557503765_f215_chunk_57_subchunk_1.wav
lahaja/audios/5348024557503765_f215_chunk_42_subchunk_1.wav
lahaja/audios/5348024557506963_f94_chunk_16_subchunk_1.wav
lahaja/audios/5348024557503487_f35_chunk_4_subchunk_1.wav
lahaja/audios/5348024557508223_f518_chunk_4_subchunk_1.wav
lahaja/audios/5348024557503647_f268_chunk_0_subchunk_1.wav
lahaja/audios/5348024557503428_f206_chunk_6_subchunk_1.wav
lahaja/audios/5348024557505404_f700_chunk_1_subchunk_1.wav
lahaja/audios/5348024557507089_f1019_chunk_0_subchunk_1.wav
lahaja/audios/5348024557507011_f710_chunk_6_subchunk_1.wav
lahaja/audios/5348024557510882_f28_chunk_2_subchunk_1.wav
lahaja/audios/5348024557506372_f470_chunk_3_subchunk_1.wav
lahaja/audios/5348024557503416_f607_chunk_0_subchunk_1.wav
lahaja/audios/5348024557508361_f101_chunk_17_subchunk_1.wav
la

In [2]:
# clone nemo
!git clone https://github.com/AI4Bharat/NeMo.git -b nemo-v2

# install NeMo
%cd NeMo
!bash reinstall.sh

Cloning into 'NeMo'...
remote: Enumerating objects: 79391, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 79391 (delta 234), reused 232 (delta 201), pack-reused 79080 (from 1)
Receiving objects: 100% (79391/79391), 140.95 MiB | 29.37 MiB/s, done.
Resolving deltas: 100% (57651/57651), done.
/content/NeMo
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Uninstalling stuff
Installing nemo
Obtaining file:///content/NeMo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metad

In [3]:
!pip install packaging
!pip install huggingface_hub==0.23.2

  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [4]:
import torch
import soundfile as sf
import nemo.collections.asr as nemo_asr
import IPython

[NeMo W 2024-09-19 09:20:15 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2024-09-19 09:20:15 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    


In [19]:
import csv
import json
import os
import librosa

def build_manifest(csv_file_path, manifest_path, audio_dir):
    os.makedirs(os.path.dirname(manifest_path), exist_ok=True)

    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        with open(manifest_path, 'w', encoding='utf-8') as manifest_file:
            for row in reader:
                audio_path = os.path.join(audio_dir, row['fname'])

                # Verify if the audio file exists
                if not os.path.isfile(audio_path):
                    print(f"Warning: Audio file not found: {audio_path}")
                    continue

                # Get duration using librosa if not provided in CSV
                duration = float(row['duration']) if 'duration' in row else librosa.get_duration(filename=audio_path)

                manifest_entry = {
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": row['normalized_cleaned']
                }

                json.dump(manifest_entry, manifest_file)
                manifest_file.write('\n')

    print(f"Manifest file created at {manifest_path}.")

# Usage
csv_file_path = '/content/meta-data-lahaja_cleaned.csv'
manifest_path = '/content/lahaja_manifest.json'
audio_dir = '/content/lahaja/audios'

build_manifest(csv_file_path, manifest_path, audio_dir)

Manifest file created at /content/lahaja_manifest.json.


In [20]:
!python /content/transcribe_speech.py \
pretrained_name="stt_hi_conformer_ctc_medium" \
audio_dir="/content/lahaja/audios/" \
dataset_manifest="/content/lahaja_manifest.json" \
output_filename="/content/lahaja_out.json" \
batch_size=10 \
calculate_wer=True \
cuda=0 \
amp=True

[NeMo W 2024-09-19 09:54:31 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2024-09-19 09:54:31 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-09-19 09:54:35 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_

In [23]:
!python /content/eval.py \
dataset_manifest="/content/lahaja_out.json" \
use_cer=False \
only_score_manifest=True

[NeMo W 2024-09-19 09:59:18 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2024-09-19 09:59:18 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-09-19 09:59:23 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_

In [22]:
!python /content/eval.py \
model_path=null \
pretrained_name="stt_hi_conformer_ctc_medium" \
dataset_manifest="/content/lahaja_manifest.json" \
output_filename="/content/lahaja_out.json" \
batch_size=32 \
amp=True \
use_cer=False

[NeMo W 2024-09-19 09:57:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2024-09-19 09:57:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-09-19 09:57:05 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_

In [ ]:

OMP_NUM_THREADS=64
!python /content/speech_to_text_eval.py \
dataset_manifest=/content/lahaja_manifest.json \
output_filename=/content/lahaja_out.json \
pretrained_name="stt_hi_conformer_ctc_medium" \
langid=hi \
batch_size=64 \
compute_timestamps=False \
compute_langs=False \
amp=True \
append_pred=False

Traceback (most recent call last):
  File "/content/speech_to_text_eval.py", line 71, in <module>
    import transcribe_speech
  File "/content/transcribe_speech.py", line 23, in <module>
    import pytorch_lightning as pl
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/__init__.py", line 27, in <module>
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/__init__.py", line 14, in <module>
    from pytorch_lightning.callbacks.batch_size_finder import BatchSizeFinder
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/batch_size_finder.py", line 26, in <module>
    from pytorch_lightning.callbacks.callback import Callback
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/callback.py", line 22, in <module>
    from pytorch_lightning.utilities.types import STEP_OUTPUT
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/types.py", line 42, in <module>
    from torchmetrics im